# Application en Flask

Aujourd'hui nous allons reporduire en partie le jeu "Big Data ou Pokemon" disponible à l'adresse suivante https://pixelastic.github.io/pokemonorbigdata/

Pour réussir cela, nous aurons besoin de plusieurs choses : 
- une liste des noms de pokemon
- une liste de techologies Big data
- un moyen de créer un site internet avec Python (et c'est là que Flask apparait)

## Récupérer la liste des noms des pokemons

L'exercice de scraping du TD "WebScraping" nous donne leur noms ainsi qu'une rapide description des pokemons

In [1]:
import pandas as pd

db_pokemon = pd.read_csv("./pokemons_database.csv", encoding="utf-8" )
dict_pokemons = db_pokemon[['name','desc']].set_index('name').to_dict()['desc']
print(list(dict_pokemons.keys())[0:10])

['Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartortle', 'Blastoise', 'Caterpie']


## Récupérer la liste des technologies Big Data

Pour cela, nous allons passer par le site http://usefulstuff.io/big-data/ qui donne une liste assez complète de ce qui existe sur le marché

In [3]:
import urllib
import bs4
import collections
import pandas as pd

req = urllib.request.Request('http://usefulstuff.io/big-data/')
html = urllib.request.urlopen(req).read()
page = bs4.BeautifulSoup(html, "lxml")


URLError: <urlopen error [Errno -2] Name or service not known>

In [3]:
dict_big_data_techno = collections.defaultdict()
for body in page.findAll('div', {'class' : 'entry-content'}) : 
    for part in body.findAll('ul') : 
        for item_list in body.findAll('li') : 
            if ":" in item_list.get_text() : 
                desc_big_data = item_list.get_text().replace("Ê",'').replace("’",'').replace(" ",'')
                dict_big_data_techno[desc_big_data.split(':')[0]] = desc_big_data

On voudra supprimer les mots qui font directement référence à une société connue dans le nom du produt (Facebook, Twitter...)

In [4]:
tags = ['Amazon','Apache','Adobe', 'Facebook','Twitter','Ebay', 'Netflix', 'LinkedIn', 'Spotify', "Cluster", 'Hive', 'Analytics',
        'Google', 'Hadoop','Linkedin', 'Pinterest', 'Cloudera', 'Cloud', 'Eclipse','Microsoft', 'SQL', 
        'DB', '.js', 'Yahoo', 'Azure', 'ML', 'Machine Learning', 'IBM', "SAP", "System", "Spark",'.']

def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        try:
            initializer = next(it)
        except StopIteration:
            raise TypeError('reduce() of empty sequence with no initial value')
    accum_value = initializer
    for x in it:
        accum_value = function(accum_value, x)
    return accum_value


corrected_dict = { reduce(lambda a,b: a.replace(b, '').replace('-',' '), tags, x) : dict_big_data_techno[x] for x in dict_big_data_techno.keys() }


final_dict_big_data_techno = { k.capitalize():v for k,v in corrected_dict.items() if " " not in k and len(k) > 4 }
print(final_dict_big_data_techno)

{'Netezza': 'IBMNetezza:high-performancedatawarehouseappliances', 'Accumulo': 'ApacheAccumulo:distribuitedkey/valuestore,builton\xa0Hadoop', 'Monet': 'MonetDB:columnstoredatabase', 'Proxy': 'ProxySQL:HighPerformanceProxyforMySQL', 'Level': 'LevelDB:afastkey-valuestoragelibrarywrittenatGooglethatprovidesanorderedmappingfromstringkeystostringvalues', 'Cosmos': 'MicrosoftCosmos:MicrosoftsinternalBigDataanalysisplatform', 'Memcache': 'MemcacheDB:adistributedkey-valuestoragesystemdesignedforpersistent', 'Stream': 'MicrosoftAzureStreamAnalytics:aneventprocessingenginethathelpsuncoverreal-timeinsightsfromdevices,sensors,infrastructure,applicationsanddata', 'Pigpen': 'NetflixPigPen:map-reduceforClojurewhichecompilestoApachePig', 'Zalonibedrock': 'ZaloniBedrock:fullyintegratedHadoopdatamanagementplatform', 'Peity': 'Peity:ProgressiveSVGbar,lineandpiecharts', 'Ayasdicore': 'AyasdiCore:toolfortopologicaldataanalysis', 'Squarecubism': 'SquareCubism.js:aD3pluginforvisualizingtimeseries.UseCubismtoc

In [5]:
df = pd.DataFrame.from_dict(final_dict_big_data_techno, orient  = 'index')
df.reset_index(inplace = True)
df.columns = ['name', 'desc']
df.to_csv("../Scraping/big_data_database.csv",encoding = "utf8")

## Préparation pour Flask

Ici on prépare le bout de code qui servira pour Flask :
- l'import des fichiers avec les noms de Pokémons et de Big data
- la création d'un dictionnaire identifiant les pokemons comme des pokemons et les technos big data comme telles

In [6]:
import random
import collections
import pandas

df_pokemon = pd.read_csv("./pokemons_database.csv")
df_big_data = pd.read_csv("./big_data_database.csv")

dict_pokemons = db_pokemon[['name','desc']].set_index('name').to_dict()['desc']
dict_big_data = df_big_data[['name', 'desc']].set_index('name').to_dict()['desc']

dict_global  = collections.defaultdict()

### On crée le dictionnaire qui regroupe les pokemons et les techno big data

for pokemon in dict_pokemons.keys() :  
    dict_global[pokemon] = "Pokemon"

for techno in dict_big_data.keys() :  
    dict_global[techno] = "Big Data"

Ici, on choisit 10 éléments Pokemon et Big Data pour réaliser le questionnaire.

Grâce à random.seed() on choisit aléatoirement parmi les noms disponibles mais le résultat sera toujours le même. 

In [45]:
r = random.Random(100)

# on choisit 10 éléments dans la liste globale des pokemons et des techno big data    
global_list = list(dict_pokemons.keys())+list(dict_big_data.keys())
liste_choice =[]

for x in range(0,11) : 
    liste_choice.append(r.choice(global_list))
for item in liste_choice : 
    print(item, dict_global[item])

Flaaffy Pokemon
Myria Big Data
Featurefu Big Data
Sceptile Pokemon
Akiban Big Data
Mime-jr Pokemon
Bookkeeper Big Data
Hparser Big Data
Chesnaught Pokemon
Velox Big Data
Lanturn Pokemon


In [41]:
liste_choice = ['Flaaffy',
 'Myria',
 'Featurefu',
 'Sceptile',
 'Akiban',
 'Mime-jr',
 'Bookkeeper',
 'Hparser',
 'Chesnaught',
 'Velox',
 'Lanturn']

In [42]:
questions = collections.defaultdict()    
for x in range(1,11) : 
    questions[str(x)] = {"question" : "{} : Pokemon or Big Data ?".format(liste_choice[x-1]), "options" :["Pokemon", "Big Data"], "answer" : dict_global[liste_choice[x-1]]}
    
print(questions)

defaultdict(None, {'3': {'options': ['Pokemon', 'Big Data'], 'question': 'Featurefu : Pokemon or Big Data ?', 'answer': 'Big Data'}, '4': {'options': ['Pokemon', 'Big Data'], 'question': 'Sceptile : Pokemon or Big Data ?', 'answer': 'Pokemon'}, '6': {'options': ['Pokemon', 'Big Data'], 'question': 'Mime-jr : Pokemon or Big Data ?', 'answer': 'Pokemon'}, '1': {'options': ['Pokemon', 'Big Data'], 'question': 'Flaaffy : Pokemon or Big Data ?', 'answer': 'Pokemon'}, '7': {'options': ['Pokemon', 'Big Data'], 'question': 'Bookkeeper : Pokemon or Big Data ?', 'answer': 'Big Data'}, '2': {'options': ['Pokemon', 'Big Data'], 'question': 'Myria : Pokemon or Big Data ?', 'answer': 'Big Data'}, '8': {'options': ['Pokemon', 'Big Data'], 'question': 'Hparser : Pokemon or Big Data ?', 'answer': 'Big Data'}, '9': {'options': ['Pokemon', 'Big Data'], 'question': 'Chesnaught : Pokemon or Big Data ?', 'answer': 'Pokemon'}, '5': {'options': ['Pokemon', 'Big Data'], 'question': 'Akiban : Pokemon or Big Dat